In [1]:
#%matplotlib widget

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
#from neural_network.cam.cams import SmoothGradCAMpp, SSCAM, ScoreCAM
from neural_network.trainer import Agent
from neural_network.utils.utils import interactive_slices,interactive_slices_masked
from skimage.transform import resize

In [4]:
agent = Agent('resnet50');

Setup configurations...
Dataset sizes - Training: 421 Validation: 47 Test: 0
Architecture [ResNet] was created
Model [LightningModel] was created


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [ ]:
agent.fit()

initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 46.2 M
---------------------------------
46.2 M    Trainable params
0         Non-trainable params
46.2 M    Total params
184.645   Total estimated model params size (MB)


In [ ]:
agent.model;

In [ ]:
val_iter = iter(agent.dataset.val_dataloader())

In [ ]:
first_image, first_label = next(val_iter)

In [ ]:
# Import M3d-CAM
from medcam import medcam

# Init your model and dataloader
model = agent.model
data_loader = agent.dataset.val_dataloader()

# Inject model with M3d-CAM
model = medcam.inject(model, backend='gcampp',output_dir="attention_maps", label=0,layer='model.layer4',save_maps=True,data_shape=(79,224,224))

model.eval()
output = model(first_image[0].unsqueeze(0))

In [ ]:
darp = medcam.medcam_utils.normalize(model.medcam_dict['current_attention_map'])
darp = resize(darp.squeeze(0).squeeze(0),(79,224,224))

In [ ]:
%matplotlib widget

In [ ]:
model.eval()
plt.close()
#for first_image, first_label in agent.dataset.val_dataloader():

output = model(first_image[0].unsqueeze(0))

asd = interactive_slices_masked()
asd.multi_slice_viewer(first_image[0],darp)
#asd.cycle(0.3)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig = plt.figure()
plt.imshow((first_image[0].squeeze(0)[50] * 255).numpy().astype(np.uint8), cmap='gray')
plt.imshow((darp[50] * 255).astype(np.uint8), cmap='jet', alpha=0.3)

plt.show()

In [ ]:
from neural_network.utils.utils import interactive_slices
testplot = interactive_slices()
testplot.multi_slice_viewer(darp)

In [ ]:
from neural_network.cam.cam import SmoothGradCAMpp

In [ ]:
wrapped_model = SmoothGradCAMpp(model,target_layer=model.model.layer4[1].conv2)

In [ ]:
first_image[0].unsqueeze(0).shape

In [ ]:
model.eval()
cam, idx = wrapped_model(first_image[0].unsqueeze(0))